In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import os

In [ ]:
def timings(path):
    '''
    Determine the walltime for a run

    parameters
    ----------
    path: str
        Path to the model_log.csv files
    
    returns
    -------
    maxiter: int
        Number of iterations ran
    datatime: pandas._libs.tslibs.timedeltas.Timedelta
        Pandas timing
    
    '''
    # Grab all of the model_log.csv files
    try:
        files = sorted(glob(f'{path}/*model_log.csv'),key=lambda x: int(x.split('/')[-1].split('_')[0]))
    except:
        files = sorted(glob(f'{path}/*model_log.csv'))
    
    datatime = pd.Timedelta(0)  # Initialize as Timedelta
    itercost = []
    statsdf = []
    # Loop over files and return the total wall time and number of iterations ran
    for idx, i in enumerate(files):
        df1 = pd.read_csv(i).dropna().drop(columns=['Parameters'])
        df = pd.to_datetime(df1["Time"], format="%a %b %d %H:%M:%S %Y")
        
        datatime+=df.iloc[-1] - df.iloc[0] 
        itercost.append((df1[['Iteration','Cost']]))
        if idx==len(files)-1:
            maxiter = int(df1['Iteration'].values[-1])
        statsdf.append(df.diff().mean())
    # Drop NaT values
    time_deltas_clean = pd.Series(statsdf).dropna()
    
    # Compute statistics
    min_time = time_deltas_clean.min()
    mean_time = time_deltas_clean.mean()
    max_time = time_deltas_clean.max()
    median_time = time_deltas_clean.median()        
    data = pd.DataFrame.from_dict({"Num_iter":maxiter,'Total Walltime':datatime,"min_time":min_time,"mean_time":mean_time,"max_time":max_time,"median_time":median_time},orient="index")
    
    try:
        label = sum([[i for i in path.split('/') if f'{j}RUD' in i or f"{j}AL" in i ] for j in [3,5,7,9]],[])[0]
        sns.lineplot(pd.concat(itercost),x='Iteration',y='Cost',label=label,alpha=0.7)
    except:
        label = os.path.basename(path)
    
    
    return data

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def process_file(file):
    try:
        return timings(file).rename(columns={0: os.path.basename(file)})
    except Exception as e:
        return None

files = sorted(glob('../BSE-calc-test/16qubit/*'))

with ThreadPoolExecutor() as executor:
    qub16 = list(executor.map(process_file, files))

In [ ]:
pd.concat(qub16,axis=1).T.sort_values(by='Num_iter')

In [ ]:
BSE5 = pd.concat([timings(i).rename(columns={0:i.split('/')[-2]}) for i in sorted(glob('../BSE_RUD_AL_test/5qubit/*/M-M-CZ_HWE-CNOT'))],axis=1)
BSE5

In [ ]:


BSE16 = pd.concat([timings(i).rename(columns={0:i.split('/')[-2]}) for i in sorted(glob( '../BSE_RUD_AL_test/16qubit/*/A2_Hadamard'))],axis=1)
BSE16

In [ ]:
[i.split('/') for i in sorted(glob('../qml_DDCC/RUD_AL/*/A2_HWE-CNOT/'))]

In [ ]:

DDCC = pd.concat([timings(i).rename(columns={0:i.split('/')[-3]}) for i in sorted(glob('../qml_DDCC/RUD_AL/*/A2_HWE-CNOT/'))],axis=1)
DDCC